# Face Clustering for Photo Albums


## Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import numpy as np
from pathlib import Path
import ipywidgets as widgets
from IPython.core.display import display
from ipywidgets import interact, interact_manual
from ipywidgets import Button, HBox, VBox, GridspecLayout, Layout
import cv2 
from matplotlib import pyplot as plt
import src.gui_controller as gui_controller

Using TensorFlow backend.


## Extract Features

In [2]:
radio = widgets.RadioButtons(
    options=['Scanned Album', 'Image(s)'],
    description='Mode:',
    disabled=False
)
information_label = widgets.Label(value="If you have already extracted the faces once for this folder you can press 'Extract Features' right away")
button_extract_faces = Button(description="Extract Faces", button_style= 'info')
button_extract_embeddings = Button(description="Extract Features", button_style = 'primary')

image_folder_box = widgets.Text(
    value=r'C:\AX',
    placeholder='Type something',
    description='Path:',
    disabled=False
)

embeddings =[]

def set_buttons_enabled(enabled):
    button_extract_faces.disabled = not enabled
    button_extract_embeddings.disabled = not enabled
    
def extract_faces(b):
    set_buttons_enabled(False)
    information_label.value = "Face Extraction is in progress. Please wait."
    gui_controller.extract_faces_from_folder(image_folder, isAlbum)
    information_label.value = "Faces have been extracted, you can start the feature extraction now"
    set_buttons_enabled(True)
    
def extract_embeddings(b):
    global embeddings
    global face_imgs
    set_buttons_enabled(False)
    information_label.value = "Feature Extraction is in progress. Please wait."
    isAlbum = False
    if radio.value == 'Scanned Album':
        isAlbum=True
    embeddings, face_imgs = gui_controller.get_embeddings_and_imgs(image_folder_box.value)
    information_label.value = "Features have been extracted, you can now start clustering"
    set_buttons_enabled(True) 
    
button_extract_faces.on_click(extract_faces)
button_extract_embeddings.on_click(extract_embeddings)
display(image_folder_box)
display(radio)
display(button_extract_faces)
display(button_extract_embeddings)
display(information_label)

Text(value='C:\\AX', description='Path:', placeholder='Type something')

RadioButtons(description='Mode:', options=('Scanned Album', 'Image(s)'), value='Scanned Album')

Button(button_style='info', description='Extract Faces', style=ButtonStyle())

Button(button_style='primary', description='Extract Features', style=ButtonStyle())

Label(value="If you have already extracted the faces once for this folder you can press 'Extract Features' rig…

## Cluster Faces

In [6]:
threshold_slider = widgets.IntSlider(
    value=30,
    min=1,
    max=100,
    step=1,
    description='Threshold:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)
@interact
def cluster(threshold = threshold_slider):
    global embeddings
    if len(embeddings)!=0:
        predictions, cluster_count, avg_imgs_per_cluster, closest_clusters = gui_controller.get_clusters(embeddings,threshold)
        print("No of Clusters: " +  "\x1b[31m "+str(cluster_count)+ "\x1b[0m")
        print("Avg Images per Cluster: " +  "\x1b[31m "+str(round(avg_imgs_per_cluster))+ "\x1b[0m" )    
        # print("Class for every Face (-1 stands for undetected)\n")
        # print("\n Clusters:")
        # print(predictions)
        # print(closest_clusters)
    else:
        print("No embeddings available, extract Features first.")
    
preview_button =widgets.Button(
    description='Show Preview',
    disabled=False,
    button_style='info', 
    tooltip='Zeigt Preview für die generierten Cluster an',
    icon='image' # (FontAwesome names without the `fa-` prefix)
)

    
save_button =widgets.Button(
    description='Save',
    disabled=False,
    button_style='success', 
    tooltip='Speichert in Preview generierte/eingetragene Infos in csv',
    icon='save' # (FontAwesome names without the `fa-` prefix)
)


out = widgets.Output(
    max_height = 100,
    layout = {'border':'1px solid black'}

)
label_info = {}
cluster_box = None
same_checkboxes = None
correct_checkboxes = None
def save(b):
    global label_info
    for label in label_info:
        info = label_info[label]
        print('cluster label: ' + str(label))
        print('name in textbox: ' + str(info[0].value))
        print('closest cluster label: ' + str(info[1]))
        print('correct checkbox value: ' + str(info[2].value))
        print('same checkbox value: ' + str(info[3].value))
        print('\n')
    print(face_imgs)

def show_preview(b):
    with out:
        out.clear_output()
        global embeddings
        if len(embeddings)!=0:
            threshold = threshold_slider.value
            predictions, cluster_count, avg_imgs_per_cluster, closest_clusters = gui_controller.get_clusters(embeddings,threshold)
            min_cluster_size  = 1
            max_images_per_plot = 3
            labels = np.unique(predictions)
            labels = labels[labels > -1]
            global label_info
            global correct_checkboxes
            all_clusters_box = []
            heading = GridspecLayout(1, 7, layout = {'overflow':'hidden'})
            heading[0,0] = widgets.HTML(value = f"<b>Predicted Cluster</b>")
            heading[0,4] = widgets.HTML(value = f"<b>Closest Cluster</b>")
            all_clusters_box.append(heading)
            for idx, label in enumerate(labels):
                image_indices = np.where(predictions == label)[0]
                closest_cluster_label = closest_clusters[label]
                closest_cluster_indices = np.where(predictions == closest_cluster_label)[0]
                if image_indices.shape[0] >= min_cluster_size:
                    cluster_box = []
                    grid = GridspecLayout(1, 7, layout = {'overflow':'hidden','align_items':'center', 'margin': '0px 0px 15px 0px'})
                    columns = min(image_indices.shape[0], max_images_per_plot)
                    for i in range(columns):
                        file = open(face_imgs[image_indices[i]], "rb")
                        image = file.read()
                        img = widgets.Image(
                            value= image,
                            format='png',
                            width=200,
                            height=200,
                        )
                        grid[0,i] = img 

                    same_cbox = widgets.Checkbox(False, description='Same?', layout = Layout(width = 'auto', justify_content = 'center'), indent = False)
                    grid[0,3] = same_cbox
                    columns = min(closest_cluster_indices.shape[0], max_images_per_plot)
                    for i in range(columns):
                        file = open(face_imgs[closest_cluster_indices[i]], "rb")
                        image = file.read()
                        img = widgets.Image(
                            value= image,
                            format='png',
                            width=200,
                            height=200,
                        )
                        grid[0,4+i] = img 
                    name = widgets.Text(
                        value=str(idx),
                        placeholder='Enter Name',
                        description='Name:',
                        disabled=False,
                        layout = {'margin':'0px 0px 10px 0px'}
                    )
                    correct_cbox = widgets.Checkbox(True, description='Correct?', indent = False, layout = {'margin': '0px 0px 0px 15px'})
                    label_info[label] = [name,closest_cluster_label,correct_cbox,same_cbox]
                    cluster_box.append(grid)
                    
                    cluster_box.append(HBox([name, correct_cbox]))
                    v_cluster_box = VBox(cluster_box)
                    v_cluster_box.layout.border = '1px solid black'
                    v_cluster_box.layout.padding = '5px'
                    all_clusters_box.append(v_cluster_box)
            cluster_box = VBox(all_clusters_box)
            display(cluster_box)
        else:
            print("No embeddings available, go back and extract Features first.")     
preview_button.on_click(show_preview)    
save_button.on_click(save)
display(preview_button)
display(out)
display(save_button)


interactive(children=(IntSlider(value=30, continuous_update=False, description='Threshold:', min=1), Output())…

Button(button_style='info', description='Show Preview', icon='image', style=ButtonStyle(), tooltip='Zeigt Prev…

Output(layout=Layout(border='1px solid black'))

Button(button_style='success', description='Save', icon='save', style=ButtonStyle(), tooltip='Speichert in Pre…

cluster label: 0
name in textbox: 0
closest cluster label: 10
correct checkbox value: True
same checkbox value: False


cluster label: 1
name in textbox: 1
closest cluster label: 60
correct checkbox value: True
same checkbox value: False


cluster label: 2
name in textbox: 2
closest cluster label: 9
correct checkbox value: True
same checkbox value: False


cluster label: 3
name in textbox: 3
closest cluster label: 19
correct checkbox value: True
same checkbox value: False


cluster label: 4
name in textbox: 4
closest cluster label: 23
correct checkbox value: True
same checkbox value: False


cluster label: 5
name in textbox: 5
closest cluster label: 8
correct checkbox value: True
same checkbox value: False


cluster label: 6
name in textbox: 6
closest cluster label: 0
correct checkbox value: True
same checkbox value: False


cluster label: 7
name in textbox: 7
closest cluster label: 50
correct checkbox value: True
same checkbox value: False


cluster label: 8
name in textbox: 8
closest